<a href="https://colab.research.google.com/github/kaleemahmed2022/NewProject/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

     |████████████████████████████████| 124 kB 5.8 MB/s 


In [2]:
import os
import torch
import numpy as np
import tensorboardX
from tqdm import tqdm

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

from network import VggVox
from dataloader import VoxLoader
from utils import Normalize, ToTensor
from torchvision.transforms import Compose


In [7]:
DATASET_PATH = '/content/'
# LOG_DIR = './'
total_epochs = 30

torch.backends.cudnn.deterministic = True
batch_size = 96

decay = 5e-4
initial_lr = 1e-2
final_lr = 1e-4

gamma = 10 ** (np.log10(final_lr / initial_lr) / (total_epochs - 1))
device = 'cuda:0'


In [8]:
TBoard = tensorboardX.SummaryWriter()

model = VggVox(num_classes=1251)
model.to(device)

transforms = Compose([
    Normalize(),
    ToTensor()
])

trainset = VoxLoader(DATASET_PATH, train=True, transform=transforms)
trainsetloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=4, shuffle=True)

testset = VoxLoader(DATASET_PATH, train=False, transform=transforms)
testsetloader = torch.utils.data.DataLoader(testset, batch_size=1, num_workers=8)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), initial_lr, 0.9, weight_decay=decay)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
# Training
for epoch_num in range(total_epochs):
    lr_scheduler.step()

    model.train()

    for iter_num, (labels, specs) in enumerate(trainsetloader):
        optimizer.zero_grad()
        labels, specs = labels.to(device), specs.to(device)
        scores = model(specs)
        loss = criterion(scores, labels)
        loss.backward()
        optimizer.step()

        # Logging
        step_num = epoch_num * len(trainsetloader) + iter_num
        TBoard.add_scalar('Metrics/train_loss', loss.item(), step_num)
        TBoard.add_scalar('Metrics/lr', lr_scheduler.get_lr()[0], step_num)


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_che

FileNotFoundError: ignored